## Montando o drive para carregar o dataset

In [ ]:
pip install prov

In [ ]:
!python -m spacy download pt_core_news_lg
!pip install mglearn

In [ ]:
from prov.model import ProvDocument, Namespace, Literal, PROV, Identifier
import datetime

g = ProvDocument(
    namespaces={
        "ex": "MonkeyPox Feelings",
        "dcterms": "http://purl.org/dc/terms/",
        "foaf": "http://xmlns.com/foaf/0.1/",
    }
)

article = g.entity("ex:article", {"dcterms:title": "MonkeyPox Feelings - Uma análise de sentimentos de tweets sobre a MonkeyPox"})

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
import os
cwd = os.getcwd()
absolute_path = "drive/MyDrive/MestradoPPGI/topicos-ds/"
os.chdir(absolute_path)

## Carregando o dataset

In [ ]:
import pandas as pd
import json

In [ ]:
with open("teste.json","r") as f:
    json_str = f'[{f.read()}]'
    obj_list = json.loads(json_str)

In [ ]:
data = pd.DataFrame.from_dict(obj_list)

In [ ]:
tweets = pd.DataFrame.from_dict(data["data"][0])

## Quantidade de Tweets Coletados, Tweet mais antigo e Tweet mais recente

In [ ]:
print(f"Quantidade Tweets: {len(tweets.index)},\nTweet mais antigo: {tweets['created_at'].min()},\nTweet mais recente: {tweets['created_at'].max()}")

In [ ]:
tweets.head()

In [ ]:
tweets["text"][1]

## Aplicando Stopwords 

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopwords_br = set(stopwords.words('portuguese'))

stopwords_br.update(["https", "t", "t.co", "twitter", "t co","damadanoite14", "RT","RT Dr","co"])


In [ ]:
from nltk.tokenize import word_tokenize
tokens = []
for each in tweets.index:
    word_tokens = word_tokenize(tweets.iloc[each]['text'])
    tokens.append(word_tokens)


In [ ]:
filtered_sentence = []
for tk in tokens:
    for w in tk:
        if w.lower() not in stopwords_br:
            filtered_sentence.append(w)

## Visualizando palavras mais recorrentes no Wordcloud

In [ ]:
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

import matplotlib.pyplot as plt
%matplotlib inline

# Start with one review:
text = " ".join(word for word in filtered_sentence)

# Create and generate a word cloud image:
wordcloud = WordCloud().generate(text)

# Display the generated image:
plt.figure(figsize=[10,5])

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## Carregando modelo SVC

In [ ]:
from model import modelo

## Classificando Tweets com o modelo SVC

In [ ]:
saida = ""
teste = modelo.fazer_previsoes(tweets, saida)

## Levantando estatísticas quantativas de Sentimento x Tweet

In [ ]:
from collections import Counter
sentimentos_dist= Counter(teste["Sentimento"])

In [ ]:
sentimentos_dist

In [ ]:
from matplotlib import pyplot as plt
x = sentimentos_dist.keys()
y = sentimentos_dist.values()

plt.bar(x,y,width = 0.5)
plt.show()


## Analise quantativa de Sentimento x Tweet com o termo "gay".

In [ ]:
teste["gays"] = (teste["Tweet"].str.lower()
                            .str.contains("gay", regex=False, na=False)
                            .astype(int))

In [ ]:
df_gays = teste.loc[teste["gays"] == 1]

In [ ]:
df_gays.head(10)

In [ ]:
qtd_sentimentos_gays = dict(Counter(df_gays["Sentimento"]))

In [ ]:
qtd_sentimentos_gays

In [ ]:
from matplotlib import pyplot as plt
x = qtd_sentimentos_gays.keys()
y = qtd_sentimentos_gays.values()

plt.bar(x,y,width = 0.5)
plt.show()